In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [2]:
batch_size = 32
epochs = 25
input_shape = (128, 128, 3)
num_classes = 10

In [3]:
train_data_dir = 'split/train'
validation_data_dir = 'split/test'

# Data augmentation for training data
train_datagen = ImageDataGenerator(
    rescale=1.0/255,        # Normalizing pixel values to [0, 1]
    rotation_range=20,      # Random rotations
    width_shift_range=0.2,  # Horizontal shift
    height_shift_range=0.2, # Vertical shift
    shear_range=0.2,        # Shearing
    zoom_range=0.2,         # Random zoom
    horizontal_flip=True,   # Random horizontal flip
    fill_mode='nearest'     # Filling missing pixels
)

# Only rescaling for validation data
validation_datagen = ImageDataGenerator(rescale=1.0/255)

# Create generators for training and validation data
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(128, 128),
    batch_size=batch_size,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(128, 128),
    batch_size=batch_size,
    class_mode='categorical'
)

Found 21719 images belonging to 10 classes.
Found 5434 images belonging to 10 classes.


In [4]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),

    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),

    Dense(num_classes, activation='softmax')  # 10 classes for 10 skin diseases
])


C:\Users\azcos5\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [5]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [6]:
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    epochs=epochs
)


Epoch 1/25


C:\Users\azcos5\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


678/678 ━━━━━━━━━━━━━━━━━━━━ 326s 472ms/step - accuracy: 0.3952 - loss: 1.9451 - val_accuracy: 0.5560 - val_loss: 1.1974
Epoch 2/25
678/678 ━━━━━━━━━━━━━━━━━━━━ 0s 248us/step - accuracy: 0.5312 - loss: 1.3412 - val_accuracy: 0.5769 - val_loss: 0.9831
Epoch 3/25


C:\Users\azcos5\anaconda3\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


678/678 ━━━━━━━━━━━━━━━━━━━━ 323s 473ms/step - accuracy: 0.5104 - loss: 1.3387 - val_accuracy: 0.5396 - val_loss: 1.2224
Epoch 4/25
678/678 ━━━━━━━━━━━━━━━━━━━━ 0s 117us/step - accuracy: 0.4375 - loss: 1.3452 - val_accuracy: 0.6154 - val_loss: 0.9706
Epoch 5/25
678/678 ━━━━━━━━━━━━━━━━━━━━ 316s 463ms/step - accuracy: 0.5402 - loss: 1.2457 - val_accuracy: 0.5425 - val_loss: 1.2369
Epoch 6/25
678/678 ━━━━━━━━━━━━━━━━━━━━ 0s 169us/step - accuracy: 0.5000 - loss: 1.4992 - val_accuracy: 0.5385 - val_loss: 1.2027
Epoch 7/25
678/678 ━━━━━━━━━━━━━━━━━━━━ 315s 462ms/step - accuracy: 0.5502 - loss: 1.1911 - val_accuracy: 0.2482 - val_loss: 3.6740
Epoch 8/25
678/678 ━━━━━━━━━━━━━━━━━━━━ 0s 157us/step - accuracy: 0.4688 - loss: 1.2040 - val_accuracy: 0.2308 - val_loss: 3.4984
Epoch 9/25
678/678 ━━━━━━━━━━━━━━━━━━━━ 316s 463ms/step - accuracy: 0.5662 - loss: 1.1480 - val_accuracy: 0.5399 - val_loss: 1.2298
Epoch 10/25
678/678 ━━━━━━━━━━━━━━━━━━━━ 0s 143us/step - accuracy: 0.5938 - loss: 1.1716 - va

In [7]:
val_loss, val_acc = model.evaluate(validation_generator)
print(f'Validation Accuracy: {val_acc:.2f}')


170/170 ━━━━━━━━━━━━━━━━━━━━ 34s 199ms/step - accuracy: 0.5793 - loss: 1.1616
Validation Accuracy: 0.56


In [8]:
model.save('skin_disease_classifier.h5')
